<img src="figs/im2.png">

# eReefs hydrodynamics for specific site

Notebook designed by **Mark Hammerton** from **AIMS** 

In [ ]:
import os
import numpy as np
import pandas as pd

import datetime as dt

import netCDF4
from netCDF4 import Dataset, num2date

from pylab import *

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In this notebook, we use OpeNDAP to extract time series data at a single location of interest, then plot this data. This extraction process can also be done with the AIMS eReefs data extraction tool (https://extraction.ereefs.aims.gov.au/). 

*Note*: This script has no error checking and so changing the date ranges or locations might result in out of bounds errors.

# Connect to the OpeNDAP endpoint for a specified month. 

We query the server based on the OPeNDAP URL for a specific file "EREEFS_AIMS-CSIRO_gbr4_v2_hydro_daily-monthly-YYYY-MM.nc". 


- **gbr4**: we use the Hydrodynamic 4km model and daily data for the month specified

In [ ]:
month = 3
year = 2020

netCDF_datestr = str(year)+'-'+format(month, '02')
print('File chosen time interval:',netCDF_datestr)

inputFile = "http://thredds.ereefs.aims.gov.au/thredds/dodsC/s3://aims-ereefs-public-prod/derived/ncaggregate/ereefs/gbr4_v2/daily-monthly/EREEFS_AIMS-CSIRO_gbr4_v2_hydro_daily-monthly-"+netCDF_datestr+".nc"

We now load the dataset within the Jupyter environment...

In [ ]:
nc_data = Dataset(inputFile, 'r')

print('Get the list of variable in the file:')
print(list(nc_data.variables.keys()))

ncdata = nc_data.variables

To get information for a specific variables, we can use the following:

In [ ]:
ncdata['mean_cur']

In [ ]:
ncdata['mean_cur'].standard_name

In [ ]:
ncdata['mean_cur'].units

# Load variables

We then load the file dataset in Jupyter

In [ ]:
# Starting with the spatial domain
lat = ncdata['latitude'][:].filled(fill_value=0.)
lon = ncdata['longitude'][:].filled(fill_value=0.)

print('eReefs model spatial extent:\n')
print(' - Longitudinal extent:',lon.min(),lon.max())
print(' - Latitudinal extent:',lat.min(),lat.max())

In [ ]:
# Get time span of the dataset
time_var = ncdata['time']

# Starting time
dtime = netCDF4.num2date(time_var[0],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print(' - start time: ',daystr,'\n')

# Ending time
dtime = netCDF4.num2date(time_var[-1],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print(' - end time: ',daystr,'\n')

ntime = len(time_var)

print(' - Number of time steps',ntime,'\n')

In [ ]:
# Number of vetical points along the z-coordinate model
zc = ncdata['zc'][:].filled(fill_value=0.)
nlay = len(zc)

print('Number of vertical layers',nlay)

for k in range(nlay):
    print(f'  + vertical layer {k} is at {zc[k]} m')

# Picking the site location

Find the closest index to the location of interest. Here we chose **Davies Reef**

+ lat = -18.82
+ lon = 147.64

In [ ]:
site_lat = -18.82
site_lon = 147.64

def find_nearest(array, value):
    '''
    Find index of nearest value in a numpy array
    '''
    
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    
    return idx


selectedLatIndex = find_nearest(lat,site_lat)
selectedLonIndex = find_nearest(lon,site_lon)

print("Grid position of location: "+str(selectedLatIndex)+", "+str(selectedLonIndex))

Extract the values over time at this location. Note that because we are access the underlying data here this results in an OpeNDAP call to get the data from the remote server. As a result this call can take a while (~10 sec).

In [ ]:
selectedDepthIndex = 15 # -1.5m
selectedDepthIndex2 = 10 # -17.75m

# Current
# Time, Depth, Lat, Lon
dailyCurr = nc_data.variables['mean_cur'][:,[selectedDepthIndex,selectedDepthIndex2], selectedLatIndex, selectedLonIndex]
print("Current: \n",dailyTemp1[0:5])

# Temperature
# Time, Depth, Lat, Lon
dailyTemp = nc_data.variables['temp'][:,[selectedDepthIndex,selectedDepthIndex2], selectedLatIndex, selectedLonIndex]
print("Temperature: \n",dailyTemp1[0:5])

# Salinity
# Time, Depth, Lat, Lon
dailySalt = nc_data.variables['salt'][:,[selectedDepthIndex,selectedDepthIndex2], selectedLatIndex, selectedLonIndex]
print("\nSalinity: \n",dailyTemp1[0:5])

Let's get the wind for the same location. The wind variable doesn't have any depth dimension and so our indexing into the data is different. The wind is a vector measurement, with an x and y component.

In [ ]:
wspeed_v = nc_data.variables['wspeed_v'][:, selectedLatIndex, selectedLonIndex]
wspeed_u = nc_data.variables['wspeed_u'][:, selectedLatIndex, selectedLonIndex]

To get the wind speed we need to calculate the magnitude of this vector.

In [ ]:
wspeed = np.sqrt(wspeed_v**2 + wspeed_u**2)

Get the time series. Note that the time values are stored as the number of days since 1990-01-01 00:00:00 +10. 

In [ ]:
times = pd.to_datetime(time_var[:],unit='D',origin=pd.Timestamp('1990-01-01'))

## Current plot

In [ ]:
fig, ax1 = plt.subplots()
fig.set_size_inches(8, 5)

ax1.set_xlabel('Recorded Time Interval')
ax1.set_ylabel('Mean current (m/s)')

ax1.plot(times[1:], dailyCurr[1:,0], color='tab:blue', lw=3, label='Depth (-1.5 m)') 
ax1.plot(times[1:], dailyCurr[1:,1], color='tab:red', lw=3, label='Depth (-17.75 m)') 

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Wind speed (m/s)', color='tab:purple')  # we already handled the x-label with ax1
ax2.plot(times[1:], wspeed[1:], lw=3, color='tab:purple', label='Wind', ls = '--')
ax2.tick_params(axis='y', labelcolor='tab:purple')

fig.legend(loc='upper right', bbox_to_anchor=(0.91, 0.91))
plt.title('Davies Reef mean current stratification with depth for March 2020',fontsize=11)
plt.gcf().autofmt_xdate()
fig.tight_layout()

plt.savefig("CurrDavieReef.png",dpi=300)
plt.show()
fig.clear()
plt.close(fig)
plt.clf()

## Temperature plot

Plotting the temperature and wind profiles for the month of March is done like this:

In [ ]:
fig, ax1 = plt.subplots()
fig.set_size_inches(8, 5)

ax1.set_xlabel('Recorded Time Interval')
ax1.set_ylabel('Temperature (deg C)')

ax1.plot(times[1:], dailyTemp[1:,0], color='tab:blue', lw=3, label='Depth (-1.5 m)') 
ax1.plot(times[1:], dailyTemp[1:,1], color='tab:red', lw=3, label='Depth (-17.75 m)') 

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Wind speed (m/s)', color='tab:purple')  # we already handled the x-label with ax1
ax2.plot(times[1:], wspeed[1:], lw=3, color='tab:purple', label='Wind', ls = '--')
ax2.tick_params(axis='y', labelcolor='tab:purple')

fig.legend(loc='upper right', bbox_to_anchor=(0.91, 0.91))
plt.title('Davies Reef temperature stratification with depth for March 2020',fontsize=11)
plt.gcf().autofmt_xdate()
fig.tight_layout()

plt.savefig("TempDavieReef.png",dpi=300)
plt.show()
fig.clear()
plt.close(fig)
plt.clf()

From this graph we can see that the surface water at Davies Reef was very warm during March 2020. 

There was a strong stratification of the temperature profile with cool water at -18 m. Around the 10th March the wind picked up for a few days, mixing the water, cooling the surface down rapidly.

## Salinity plot

Same can be done with the salinity plot:

In [ ]:
fig, ax1 = plt.subplots()
fig.set_size_inches(8, 5)

ax1.set_xlabel('Recorded Time Interval')
ax1.set_ylabel('Salinity (PSU)')

ax1.plot(times[1:], dailySalt[1:,0], color='tab:blue', lw=3, label='Depth (-1.5 m)') 
ax1.plot(times[1:], dailySalt[1:,1], color='tab:red', lw=3, label='Depth (-17.75 m)') 

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Wind speed (m/s)', color='tab:purple')  # we already handled the x-label with ax1
ax2.plot(times[1:], wspeed[1:], lw=3, color='tab:purple', label='Wind', ls = '--')
ax2.tick_params(axis='y', labelcolor='tab:purple')

fig.legend(loc='lower right', bbox_to_anchor=(0.91, 0.2))
plt.title('Davies Reef salinity stratification with depth for March 2020',fontsize=11)
plt.gcf().autofmt_xdate()
fig.tight_layout()

plt.savefig("SalinityDavieReef.png",dpi=300)

plt.show()
fig.clear()
plt.close(fig)
plt.clf()